In [3]:
import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def read_schedule_txt(file_path):
    """
    Read schedule txt file and return content as string
    """
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    return text

def parse_schedule_text(schedule_text):
    tasks = []
    current_job = None

    lines = schedule_text.splitlines()

    for line in lines:
        line = line.strip()


        # STOP when reaching machine utilization section
        if line.startswith("MACHINE UTILIZATION"):
            break


        # Detect JOB line
        job_match = re.match(r"JOB\s+(\d+)", line)
        if job_match:
            current_job = int(job_match.group(1))
            continue


        # Skip headers / separators
        if (
            not line
            or line.startswith("─")
            or line.startswith("=")
            or line.startswith("Op")
            or line.startswith("-")
        ):
            continue


        # Parse operation row
        parts = line.split()
        if len(parts) >= 6 and current_job is not None:
            op = int(parts[0])
            machine = int(parts[1].replace("M", ""))
            start = float(parts[2])
            end = float(parts[3])
            duration = float(parts[4])
            power = float(parts[5])


            tasks.append({
                "job": current_job,
                "op": op,
                "machine": machine,
                "start": start,
                "end": end,
                "duration": duration,
                "power": power
            })


    return tasks


def plot_gantt_from_tasks(tasks, title="Gantt Chart"):
    machines = sorted(set(t["machine"] for t in tasks))
    machine_y = {m: i for i, m in enumerate(machines)}


    jobs = sorted(set(t["job"] for t in tasks))
    cmap = plt.cm.get_cmap("tab20", len(jobs))
    job_color = {job: cmap(i) for i, job in enumerate(jobs)}


    fig, ax = plt.subplots(figsize=(16, 7))


    for t in tasks:
        y = machine_y[t["machine"]]


        ax.barh(
            y=y,
            width=t["end"] - t["start"],
            left=t["start"],
            height=1,
            color=job_color[t["job"]],
            edgecolor="black"
        )


        ax.text(
            t["start"] + (t["end"] - t["start"]) / 2,
            y,
            f"J{t['job']}-O{t['op']}",
            ha="center",
            va="center",
            fontsize=7,
            color="black"
        )


    ax.set_yticks(list(machine_y.values()))
    ax.set_yticklabels([f"M{m}" for m in machines])
    ax.set_xlabel("Time")
    ax.set_ylabel("Machine")
    ax.set_title(title)


    legend_patches = [
        mpatches.Patch(color=job_color[j], label=f"Job {j}")
        for j in jobs
    ]
    ax.legend(
        handles=legend_patches,
        title="Jobs",
        bbox_to_anchor=(1.02, 1),
        loc="upper left"
    )


    ax.grid(True, axis="x", linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()


# 1) Read txt file
schedule_text = read_schedule_txt("CP scheduling.txt")


# 2) Parse
tasks = parse_schedule_text(schedule_text)


# 3) (Optional) kiểm tra nhanh
print("Total operations:", len(tasks))
print(tasks[:5])


# 4) Plot gantt
plot_gantt_from_tasks(tasks, title="Gantt Chart - CP - 20 Jobs")
